In [3]:
import pandas as pd
from ENV import *
import gym
import numpy as np
import warnings

# Suppress FutureWarning and UserWarning
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

algorithm_name = 'ppo'
space_mode = 'discrete'

if algorithm_name in ['a2c']:
  INTERVAL = 1000
  EPISODES = 2
else:
  INTERVAL = 1
  EPISODES = 200

fn = 'btc22clean'
data_source = f'../data/{fn}.csv'
df = pd.read_pickle(data_source).reset_index(level=0)

In [ ]:
# Callback
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import TensorBoardOutputFormat

class SummaryWriterCallback(BaseCallback):
    def _on_training_start(self):
        self._log_freq = 1  # log every INTERVAL calls

        output_formats = self.logger.output_formats
        # Save reference to tensorboard formatter object
        # note: the failure case (not formatter found) is not handled here, should be done with try/except.
        self.tb_formatter = next(formatter for formatter in output_formats if isinstance(formatter, TensorBoardOutputFormat))

    def _on_step(self) -> bool:
        # Log my_custom_reward every _log_freq(th) to tensorboard for each environment
        if self.n_calls % self._log_freq == 0:
            if vars(vars(vars(self.locals['env'])['envs'][0])['gym_env'])['current_step'] == MAX_STEPS - 1:
              total_reward = vars(vars(self.locals['env'])['envs'][0])['total_reward']
              mdd = vars(vars(self.locals['env'])['envs'][0])['mdd']
              sortino = vars(vars(self.locals['env'])['envs'][0])['sortino']
              return_ = vars(vars(self.locals['env'])['envs'][0])['net_worth'] / INITIAL_ACCOUNT_BALANCE
              # print(total_reward)
            # rewards = self.locals['my_custom_info_dict']['my_custom_reward']
            # for i in range(self.locals['env'].num_envs):
              self.tb_formatter.writer.add_scalar("rewards", total_reward, self.n_calls)
              self.tb_formatter.writer.add_scalar("mdd", mdd, self.n_calls)
              self.tb_formatter.writer.add_scalar("sortino", sortino, self.n_calls)
              self.tb_formatter.writer.add_scalar("return_", return_, self.n_calls)

In [ ]:

class TradingAgent(gym.env):
  metadata = {'render.modes': ['human']}

  def __init__(self, df, algorithm_name, space_mode):
    super().__init__()

    self.df = df
    self.df['return'] = 0
    self.df['mdd'] = 0
    
    self.algorithm_name = algorithm_name
    self.space_mode = space_mode # discrete or continuous

    # ACTION space
    if self.space_mode == 'discrete':
      # long, short, hold, close = 4
      self.action_space = gym.Space.Discrete(len(ACTIONS)) 
    else:
      # array[0] ACTIONS
      # array[1] AMOUNT - portion of portfolio, max is 1
      # array[2] LEVERAGE - later
      # array[3] SET_TP - later
      # array[4] SET_SL - later
      self.action_space = gym.Space.Box(low=np.array([0,0]), high=np.array([len(ACTIONS),1]), dtype=np.float16)

    # OBSERVATION space
      if self.space_mode == 'discrete':
        # arr[0] CLOSE - last close in value in log USDT
        # arr[1] VOLUME - in log
        # arr[2] DELTA - close / open in portion
        # arr[3] AMPLITUDE - high / low in portion
        # arr[4] CLOSE_OVER_LOW close / low in portion
        self.observation_space = gym.Space.Box(
          #       0   1   2   3   4
          low= [-30,-30, -1,  0,  0][:FEATURES], 
          high=[ 30, 30, 99, 99, 99][:FEATURES],
          shape=(WINDOW_SIZE,), 
          dtype=np.float16)

    self.mdd = 0
    self.mdd_last_sell = 0
    self.current_reward = 0
    self.total_reward = 0
    self.current_step = 0

  def _next_observation(self):
    # Get data points for the last WINDOW_SIZE 

    pass
